## Scenario 2: Train and deploy with MLFlow and AML


Doc: https://microsoft-my.sharepoint.com/:w:/p/osomorog/Ed7l1SLKac9Irz_PY-XnXaQB90-WeAosazcQOT24PRd3-g?e=hKS09V 

## Move to AML by setting the tracking URI in the backend (not in my training code), and using MLFlow CLI. 

In [1]:
!az extension add -n ml -y 

Extension 'ml' is already installed.


In [ ]:
!az login

In [1]:
import subprocess

#Get MLFLow UI through the Azure ML CLI v2 and convert to string
MLFLOW_TRACKING_URI = subprocess.run(["az", "ml", "workspace", "show", "--query", "mlflow_tracking_uri", "-o", "tsv"], stdout=subprocess.PIPE, text=True)
MLFLOW_TRACKING_URI = str(MLFLOW_TRACKING_URI.stdout).strip()

## Make sure the MLFLow URI looks something like this: 
## azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/<Sub-ID>/resourceGroups/<RG>/providers/Microsoft.MachineLearningServices/workspaces/<WS>
print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

MLFlow Tracking URI: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/providers/Microsoft.MachineLearningServices/workspaces/mlflowworkspace


In [2]:
## Set the MLFLOW TRACKING URI
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Train baseline model and log / autolog with MLFlow and submit job with MLFLow CLI

In [4]:
backend_config = {"USE_CONDA": False}

In [5]:
import mlflow
local_env_run = mlflow.projects.run(uri="simple_project", 
                                    parameters={"alpha":0.2},
                                    experiment_name="Scenario2_project",
                                    backend = "azureml",
                                    use_conda=True,
                                    backend_config = backend_config, 
                                    )

Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2021/12/06 18:13:19 INFO mlflow.projects.utils: === Created directory /tmp/tmp3n9i99vy for downloading remote URIs passed to arguments of type 'path' ===
Class AzureMLSubmittedRun: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2021/12/06 18:13:59 INFO mlflow.projects: === Run (ID 'Scenario2_project_1638814400_6d76b34a') succeeded ===


06/12/2021 18:13:17 INFO azureml.mlflow: === Creating conda environment from Mlproject for local run ===
RunId: Scenario2_project_1638814400_6d76b34a
Web View: https://ml.azure.com/runs/Scenario2_project_1638814400_6d76b34a?wsid=/subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourcegroups/osomorog/workspaces/mlflowworkspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/70_driver_log.txt

[2021-12-06T18:13:29.561970] Entering context manager injector.
[2021-12-06T18:13:29.928419] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError', 'UserExceptions:context_managers.UserExceptions'], invocation=['python train.py 0.2'])
Script type = COMMAND
[2021-12-06T18:13:29.930357] Command=python train.py 0.2
[2021-12-06T18:13:29.930561] Entering Run History Context Manager.
[2021-12-06T18:13:31.047226] Command Worki

### Download or retrieve the model from the run for testing

In [6]:
from mlflow.entities import ViewType
experiment_name="Scenario2_project"
current_experiment=mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=current_experiment.experiment_id, run_view_type=ViewType.ALL)

In [7]:
last_run_id = runs.tail(1)["run_id"].tolist()[0]
last_run_id

'Scenario2_project_1638814400_6d76b34a'

### Download and Load Test Data from JSON

In [8]:
from mlflow.tracking.client import MlflowClient
client = MlflowClient()
client.download_artifacts(last_run_id,"model/input_example.json",".")

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/osomorog4/code/Users/osomorog/MLFlow_Scenarios/Scenario2/Scenario2a_MLFlow_Only/model/input_example.json'

In [9]:
import json

with open('model/input_example.json') as f:
   sample_data = json.load(f)

#columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
print(sample_data)

{'inputs': [[0.0126481372762872, 0.0506801187398187, 0.00241654245523897, 0.0563010619323185, 0.0273260502020124, 0.0171618818193638, 0.0412768238419757, -0.0394933828740919, 0.00371173823343597, 0.0734802269665584], [-0.107225631607358, -0.044641636506989, -0.0773415510119477, -0.0263278347173518, -0.0896299427450836, -0.0961978613484469, 0.0265502726256275, -0.076394503750001, -0.0425721049227942, -0.0052198044153011], [0.0271782910803654, 0.0506801187398187, -0.0353068801305926, 0.0322009670761646, -0.0112006298276192, 0.00150445872988718, -0.0102661054152432, -0.00259226199818282, -0.0149564750249113, -0.0507829804784829], [-0.00551455497881059, 0.0506801187398187, 0.00133873038135806, -0.0848566365108683, -0.0112006298276192, -0.0166581520539057, 0.0486400994501499, -0.0394933828740919, -0.0411803851880079, -0.0880619427119953], [0.0671362140415805, 0.0506801187398187, 0.0207393477112143, -0.00567061055493425, 0.0204462859110067, 0.0262431872112602, -0.0029028298070691, -0.0025922

In [12]:
model_path = "model"
artifact_uri = "runs:/{}/{}".format(last_run_id,model_path)
model = mlflow.sklearn.load_model(artifact_uri)

In [13]:
model.predict(sample_data["inputs"])

array([149.92210841,  99.36325181, 126.288127  ,  85.10695952,
       151.26791908])

## Register Model with MLFLow

In [14]:
mlflow.register_model(artifact_uri,"scenario2model")

Registered model 'scenario2model' already exists. Creating a new version of this model...
2021/12/06 18:15:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: scenario2model, version 2
Created version '2' of model 'scenario2model'.


<ModelVersion: creation_timestamp=1638814541944, current_stage='None', description='', last_updated_timestamp=1638814541944, name='scenario2model', run_id='Scenario2_project_1638814400_6d76b34a', run_link='', source='azureml://experiments/Scenario2_project/runs/Scenario2_project_1638814400_6d76b34a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

## (Optional) Transistion Model to Production Stage

In [30]:
client = MlflowClient()
client.transition_model_version_stage(
    name="scenario2model",
    version=1,
    stage="Production"
)

<ModelVersion: creation_timestamp=1638588486086, current_stage='Production', description='', last_updated_timestamp=1638588487087, name='scenario2model', run_id='Scenario2_project_1638586862_9ed4fdf5', run_link='', source='azureml://experiments/Scenario2_project/runs/Scenario2_project_1638586862_9ed4fdf5/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

## List Model details

In [15]:
from pprint import pprint
client = MlflowClient()
for mv in client.search_model_versions("name='scenario2model'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1638814541944,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1638814541944,
    'name': 'scenario2model',
    'run_id': 'Scenario2_project_1638814400_6d76b34a',
    'run_link': '',
    'source': 'azureml://experiments/Scenario2_project/runs/Scenario2_project_1638814400_6d76b34a/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}
{   'creation_timestamp': 1638588486086,
    'current_stage': 'Production',
    'description': '',
    'last_updated_timestamp': 1638588487087,
    'name': 'scenario2model',
    'run_id': 'Scenario2_project_1638586862_9ed4fdf5',
    'run_link': '',
    'source': 'azureml://experiments/Scenario2_project/runs/Scenario2_project_1638586862_9ed4fdf5/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


## Deploy to AzureML's MIR with MLFLow
As of Dec 2021, TensorSpec for Deployment in AML is not fully supported

In [ ]:
from mlflow.deployments import get_deploy_client
import mlflow
import mlflow.sklearn

# set the tracking uri as the deployment client
client = get_deploy_client(mlflow.get_tracking_uri())

# set the model path 
# model_path = "model"
# run_id= "13c5faef-788f-439d-ba6c-cb8d280e708d"

# Retrieve model from registry
model_name = "scenario2model"
model_version = 1
model_uri = 'models:/{}/{}'.format(model_name, model_version)

# define the model path and the name is the service name
# the model gets registered automatically and a name is autogenerated using the "name" parameter below 
# set the deployment config
deploy_path = "deployment_config_v2.json"
test_config = {'deploy-config-file': deploy_path}

client.create_deployment(model_uri=model_uri,
                         config=test_config,
                         name="mlflowscenario3")

## Test endpoint with MLFlow

Find code snippet below, in the Endpoint UI in AzureML and navigate to the Consume Tab for the Endpoint you just deployed

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = sample_data

body = str.encode(json.dumps(data))

url = 'https://mlflowscenario2.westus.inference.ml.azure.com/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))